In [2]:
import xarray as xr
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter  # Custom labeling
import matplotlib.dates as mdates

from dask.distributed import Client

## Open data

In [3]:
ds_rm = xr.open_mfdataset('/media/silsbelab/LaCie/OCCCI/V5/daily/9km2/rm/chl_*', 
                          engine='netcdf4', concat_dim='time',  combine='nested').chlor_a.stack(z=('lat', 'lon'))

ds = xr.open_mfdataset('/media/silsbelab/LaCie/OCCCI/V5/daily/9km2/chl_*', 
                        engine='netcdf4', concat_dim='time', combine='nested').chlor_a.stack(z=('lat', 'lon'))

ds

<xarray.DataArray 'chlor_a' (time: 8390, z: 37324800)>
dask.array<reshape, shape=(8390, 37324800), dtype=float32, chunksize=(1, 37324800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2020-12-31
  * z        (z) MultiIndex
  - lat      (z) float64 89.98 89.98 89.98 89.98 ... -89.98 -89.98 -89.98 -89.98
  - lon      (z) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 179.9 180.0
Attributes:
    long_name:            Chlorophyll-a concentration in seawater (not log-tr...
    units:                milligram m-3
    ancillary_variables:  chlor_a_log10_rmsd chlor_a_log10_bias
    grid_mapping:         crs
    parameter_vocab_uri:  http://vocab.ndg.nerc.ac.uk/term/P011/current/CHLTVOLU
    standard_name:        mass_concentration_of_chlorophyll_a_in_sea_water
    units_nonstandard:    mg m^-3

### Rechunk along time dimension

In [4]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34825 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 33.34 GB


In [6]:
ds = ds.chunk({'z': 1000, 'time': -1}) 
ds_rm = ds_rm.chunk({'z': 1000, 'time': -1}) 
ds

<xarray.DataArray 'chlor_a' (time: 8390, z: 37324800)>
dask.array<rechunk-merge, shape=(8390, 37324800), dtype=float32, chunksize=(8390, 1000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2020-12-31
  * z        (z) MultiIndex
  - lat      (z) float64 89.98 89.98 89.98 89.98 ... -89.98 -89.98 -89.98 -89.98
  - lon      (z) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 179.9 180.0
Attributes:
    long_name:            Chlorophyll-a concentration in seawater (not log-tr...
    units:                milligram m-3
    ancillary_variables:  chlor_a_log10_rmsd chlor_a_log10_bias
    grid_mapping:         crs
    parameter_vocab_uri:  http://vocab.ndg.nerc.ac.uk/term/P011/current/CHLTVOLU
    standard_name:        mass_concentration_of_chlorophyll_a_in_sea_water
    units_nonstandard:    mg m^-3

## Preprocessing to fill NAs and smooth
#### Take data from average of surround pixels (3x3) if missing and log2 transform

In [7]:
ds = ds.fillna(ds_rm)
ds = np.log2(ds)
ds

<xarray.DataArray 'chlor_a' (time: 8390, z: 37324800)>
dask.array<log2, shape=(8390, 37324800), dtype=float32, chunksize=(8390, 1000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2020-12-31
  * z        (z) MultiIndex
  - lat      (z) float64 89.98 89.98 89.98 89.98 ... -89.98 -89.98 -89.98 -89.98
  - lon      (z) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 179.9 180.0

#### Interpolation NAs and  2 week running mean to smooth

In [8]:
ds = ds.interpolate_na(dim="time", method="linear")

ds = ds.rolling(time=14, center=True).mean() 

## Write Dataset

In [ ]:
dates, datasets = zip(*ds.groupby("time"))
dates = np.datetime_as_string(dates, unit='D')

paths = [f"/media/silsbelab/LaCie/occci/v5/daily/interp/chl_{y}.nc" for y in dates]

xr.save_mfdataset(datasets, paths)